In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 16.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=09de20061011e0b940703ef0b7d78a093a972f2c6d8a2cccbcbab44661f5f0f8
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 14.2 MB/s 
     |████████████████████████████████| 3.4 MB 13.3 MB/s 
     |████████████████████████████████| 596 kB 67.1 MB/s 
     |████████████████████████████████| 895 kB 66.0 MB/s 
     |████████████████████████████████| 61 kB 624 kB/s 
     |████████████████████████████████| 3.3 MB 56.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-8v395c98
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-8v395c98


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2022-01-07 05:27:38--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6023:18::a27d:4312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2022-01-07 05:27:39--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1d9daad7d3b8ce6f896ec36831.dl.dropboxusercontent.com/cd/0/get/BdQHhXejvuKh2U07EapC7QKOaQaapK8LY65aqRdN6kB4oYOwaSnj0vaFwx8NHTq4PfcpYXp13dt4ZzJnchA7RNThVwIM5ZgfjuMuyPivAs3y18ZY36ZfwJuh3WTWKkjc8FbIFrlB4hx_OzrMbEh4CLtt/file?dl=1# [following]
--2022-01-07 05:27:39--  https://uc1d9daad7d3b8ce6f896ec36831.dl.dropboxusercontent.com/cd/0/get/BdQHhXejvuKh2U07EapC7QKOaQaapK8LY65aqRdN6kB4oYOwaSnj0vaFwx8NHTq4PfcpYX

In [ ]:
## GPU
device = torch.device("cuda:0")
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
dataset_train = nlp.data.TSVDataset('ratings_train.txt?dl=1', field_indices=[1,2],num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('ratings_test.txt?dl=1', field_indices=[1,2],num_discard_samples=1)


- baseline model 불러오기 

In [ ]:
device = torch.device("cuda:0")

model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
base_md = torch.load('/content/drive/MyDrive/final_project/kobert_v1.pt',map_location=device)
model.load_state_dict(model.state_dict())
model.eval()


## 2. 파라미터 설정하기
기계 학습을 훈련시킬 때 필요한 파라미터 설정  
- max_len : 텍스트 데이터 최대 길이
- num_epochs : 훈련 반복 횟수

In [ ]:
## setting parameters

max_len=64
batch_size=64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval =200
learning_rate = 5e-5


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

device = torce.device

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/2344 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7470998167991638 train acc 0.484375
epoch 1 batch id 201 loss 0.5169434547424316 train acc 0.5680192786069652
epoch 1 batch id 401 loss 0.4554039239883423 train acc 0.6752259975062345
epoch 1 batch id 601 loss 0.41397252678871155 train acc 0.7266794925124792
epoch 1 batch id 801 loss 0.4378545582294464 train acc 0.7566128277153558
epoch 1 batch id 1001 loss 0.3194911479949951 train acc 0.775255994005994
epoch 1 batch id 1201 loss 0.47969111800193787 train acc 0.7880932556203164
epoch 1 batch id 1401 loss 0.35937240719795227 train acc 0.7980125802997858
epoch 1 batch id 1601 loss 0.33873775601387024 train acc 0.8070346658338539
epoch 1 batch id 1801 loss 0.2966946065425873 train acc 0.8136278456413104
epoch 1 batch id 2001 loss 0.3944935202598572 train acc 0.8196448650674663
epoch 1 batch id 2201 loss 0.31629109382629395 train acc 0.8245044865970014
epoch 1 train acc 0.828287204920364


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.8969789002557544


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.3998437523841858 train acc 0.890625
epoch 5 batch id 201 loss 0.05028539523482323 train acc 0.9745802238805971
epoch 5 batch id 401 loss 0.0721968486905098 train acc 0.9753350997506235
epoch 5 batch id 601 loss 0.12196473032236099 train acc 0.9756915557404326
epoch 5 batch id 801 loss 0.05889178812503815 train acc 0.9762211298377028
epoch 5 batch id 1001 loss 0.014923879876732826 train acc 0.9768824925074925
epoch 5 batch id 1201 loss 0.016611890867352486 train acc 0.9771674646128227
epoch 5 batch id 1401 loss 0.015298038721084595 train acc 0.9778506423982869
epoch 5 batch id 1601 loss 0.06658408790826797 train acc 0.9781386633354153
epoch 5 batch id 1801 loss 0.09299857914447784 train acc 0.9785709328151028
epoch 5 batch id 2001 loss 0.08674609661102295 train acc 0.9788074712643678
epoch 5 batch id 2201 loss 0.09509172290563583 train acc 0.9790578146297138
epoch 5 train acc 0.9792422141638225


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 5 test acc 0.8972186700767263


In [ ]:
torch.save(model,'kobert_v1.pt')